In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import sklearn
from sklearn import metrics
import seaborn as sns
from sklearn.metrics import mean_squared_error


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import os
from google.colab import drive

# Mount google drive
DRIVE_MOUNT='/content/gdrive'
drive.mount(DRIVE_MOUNT)

# create folder to write data to
CIS545_FOLDER=os.path.join(DRIVE_MOUNT, 'My Drive', 'CIS545_2020')
HOMEWORK_FOLDER=os.path.join(CIS545_FOLDER, 'Project')
os.makedirs(HOMEWORK_FOLDER, exist_ok=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
x_train = pd.read_csv('/content/gdrive/My Drive/CIS545_2020/Project/training_data.csv')
x_test = pd.read_csv('/content/gdrive/My Drive/CIS545_2020/Project/testing_data.csv')
y_train = pd.read_csv('/content/gdrive/My Drive/CIS545_2020/Project/train_labels.csv')
y_test = pd.read_csv('/content/gdrive/My Drive/CIS545_2020/Project/test_labels.csv')

In [ ]:
x_train.drop(columns=['Unnamed: 0'], inplace = True)
x_test.drop(columns=['Unnamed: 0'], inplace = True)
y_train.drop(columns=['Unnamed: 0'], inplace = True)
y_test.drop(columns=['Unnamed: 0'], inplace = True)

In [ ]:
fake = pd.read_csv('/content/gdrive/My Drive/CIS545_2020/Project/fakeForModeling.csv')
true = pd.read_csv('/content/gdrive/My Drive/CIS545_2020/Project/trueForModeling.csv')

In [ ]:
fake.drop(columns=['Unnamed: 0'], inplace = True)
true.drop(columns=['Unnamed: 0'], inplace = True)

Random Forest

In [ ]:
#Tried depths greater and less than 60, 60 was the best. 
#Tried estimators greater and less than 200 but 200 was the best

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier(random_state = 1)

param_grid = {
    'n_estimators': [200],
    'max_depth': [50, 60, 70]
    }

%time grid_search_rf = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 5)
grid_search_rf.fit(x_train, np.ravel(y_train))
grid_search_rf.best_params_

CPU times: user 116 µs, sys: 0 ns, total: 116 µs
Wall time: 119 µs


{'max_depth': 50, 'n_estimators': 200}

In [ ]:
y_pred_rf = grid_search_rf.predict(x_test)

In [ ]:
print("Accuracy is:", metrics.accuracy_score(y_test, y_pred_rf))
print("Mean Squared Error is:", np.sqrt(mean_squared_error(y_test, y_pred_rf)))

Accuracy is: 0.9465478841870824
Mean Squared Error is: 0.23119713625587493


In [ ]:
rf_cm = metrics.confusion_matrix(y_test, y_pred_rf)
print(rf_cm)

[[4028  287]
 [ 193 4472]]


In [ ]:
print(metrics.classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.95      0.93      0.94      4315
           1       0.94      0.96      0.95      4665

    accuracy                           0.95      8980
   macro avg       0.95      0.95      0.95      8980
weighted avg       0.95      0.95      0.95      8980



In [ ]:
# Calculate predicted probabilities for test data 
y_pred_prob_rf = grid_search_rf.predict_proba(x_test)[:, 1]

In [ ]:
metrics.roc_auc_score(y_test, y_pred_prob_rf)

0.9870667516167212